# Overview
This notebook demonstrates usage of the MetaDoE Particle Swarm Optimization implementation by optimizing the Rastrigin and Griewank functions under constraint.

In [2]:
include("../src/doe/designs.jl")
using .Designs: hypercube_initializer

include("../src/optim/objectives.jl")
using .Objectives: rastrigin, griewank

include("./pso.jl")
using .PSO: initialize_swarm, optimize, get_optimizer, aggregate_results

In [3]:
N = 7
K = 3

3

In [5]:
initializer = hypercube_initializer(N, K; lower=-10, upper=10)
swarm = initialize_swarm(initializer, griewank)
final_swarm, history = optimize(swarm, aggregate_results(save_world=true))

(Main.PSO.PSOTypes.RunnerState(Main.PSO.PSOTypes.Swarm(Main.PSO.PSOTypes.ParticleState([-4.629064633864186e-9 8.062026204092937e-9 … 3.6785957678766233e-9 6.011469167782573e-10; 3.0233904105559574e-9 6.57598484438021e-9 … 3.087732433211301e-10 -3.956968299916313e-9; … ; 1.48711259512585e-9 7.1883216626159275e-9 … -6.124512071133266e-9 -1.5709588381537555e-8; 1.5033264668721966e-9 5.36003654022499e-9 … 3.490285645164396e-9 1.3738471159625708e-8;;; -1.703469895866257e-10 -9.373395314264086e-10 … -9.348271519304883e-11 4.526950895994493e-10; -5.048371163987062e-10 -4.3419166403957755e-9 … -1.9706704734822538e-9 -1.1691874817254928e-10; … ; 2.1461453816843864e-9 4.760309742295524e-10 … -1.9613727485973712e-10 -7.002752248366162e-9; -2.1617301759886605e-9 -2.8435089295544233e-9 … 5.568706510934746e-9 -6.480056805661855e-10;;; -7.81903349381602e-11 -5.765073153778229e-9 … -2.6975732582331323e-8 6.636007273868345e-10; -3.2698204482656892e-9 -1.4213113937543764e-8 … -2.2386959056186386e-8 1.94

In [6]:
history

501-element Vector{Any}:
 Main.PSO.PSOTypes.Swarm(Main.PSO.PSOTypes.ParticleState([5.087460100114787e-9 7.034645460519848e-9 … 2.63782488544456e-9 6.244398713282809e-9; -1.4643422713852325e-9 5.86438231762571e-9 … -1.1019573632066037e-9 -3.683984944515396e-9; … ; -1.5245519719386123e-9 -1.210277360702412e-8 … -1.229298210062745e-8 -1.031671634223911e-8; -2.022551605682052e-9 1.6792120013021542e-9 … 2.675948379613944e-9 1.7062845589571306e-8;;; -2.5609933062706876e-10 -4.910207263683253e-10 … -2.1326527024174884e-9 3.1792584162950283e-10; -2.097825272627019e-9 -6.136190390370116e-9 … -3.265965134183542e-9 -1.2207711824612838e-9; … ; 5.024678611415055e-9 -2.0864090911783127e-8 … 2.430188858310246e-9 -1.4019566747387655e-8; -2.9146130153373578e-9 -7.587990271500564e-9 … 9.547701554405932e-9 -1.6197745887568998e-9;;; -2.8451405820795226e-10 -8.328054431016071e-9 … -1.9393172690716202e-8 4.934813465351707e-10; -2.69577776717904e-9 -7.284109114718462e-9 … -1.5467792776184663e-8 7.70528498573